In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from torchtext import vocab, data, datasets

### Weighted NB

In [40]:
sl=1000
vocab_size=200000

In [41]:
PATH='tmp/aclImdb/'

names = ['neg','pos']
trn,trn_y = texts_from_folders(f'{PATH}train',names)
val,val_y = texts_from_folders(f'{PATH}test',names)

In [3]:
PATH='tmp/rotten_imdb/'
names = ['plot.tok.gt9.5000','quote.tok.gt9.5000']
x,y = texts_from_files(f'{PATH}', names)
trn,val,trn_y,val_y = train_test_split(x,y, test_size=0.2, random_state=42)

In [42]:
veczr =  CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=vocab_size)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [43]:
md = TextClassifierData.from_bow(trn_term_doc, trn_y, val_term_doc, val_y, sl)

subjective

In [39]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-5, cycle_len=1)

A Jupyter Widget

A Jupyter Widget

[ 0.067075  0.18288   0.925781]



imdb sentiment

In [54]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-6)

A Jupyter Widget

A Jupyter Widget

[ 0.058516  0.106984  0.924409]


### unigram

In [9]:
veczr =  CountVectorizer(tokenizer=tokenize)#,max_features=10000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

AttributeError: 'Example' object has no attribute 'lower'

In [ ]:
x=trn_term_doc
y=trn_y

p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [ ]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

In [ ]:
pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

In [ ]:
m = LogisticRegression(C=0.1, fit_intercept=False, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

In [ ]:
preds = m.predict(val_term_doc)
(preds==val_y).mean()

### bigram

In [20]:
veczr =  CountVectorizer(ngram_range=(1,2), tokenizer=tokenize)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [21]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [22]:
r

matrix([[-0.18822296,  1.06454001, -0.03407228, ..., -0.72721946,
         -0.72721946, -0.72721946]])

In [ ]:
x_nb = x.multiply(r)
m = LogisticRegression(dual=True, C=1, fit_intercept=False)
m.fit(x_nb, y);

In [ ]:
beta=0.25

val_x_nb = val_x.multiply(r)
w = (1-beta)*m.coef_.mean() + beta*m.coef_.T
preds = (val_x_nb @ w + m.intercept_)>0
(preds.T==val_y).mean()

In [ ]:
val_x_nb = val_x.multiply(r)
w = m.coef_.T
preds = (val_x_nb @ w + m.intercept_)>0
(preds.T==val_y).mean()

In [ ]:
w2 = w.T[0]*r.A1

In [ ]:
preds = (val_x @ w2 + m.intercept_)>0
(preds.T==val_y).mean()

In [ ]:
m = LogisticRegression(C=0.1, fit_intercept=False)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

### FastText

In [4]:
class TextDataLoader():
    def __init__(self, src, x_fld, y_fld):
        self.src,self.x_fld,self.y_fld = src,x_fld,y_fld
        
    def __len__(self): return len(self.src)
    
    def __iter__(self): 
        try:
            for b in iter(self.src):
                yield getattr(b, self.x_fld), getattr(b, self.y_fld).float()
        except StopIteration: return

def create_td(it): return TextDataLoader(it, 'text', 'label')

def postproc(arr,vocab,train): 
    return [1 if o==1 else 0 for o in arr]

In [5]:
TEXT = data.Field(lower=True, fix_length=sl)
LABEL = data.Field(sequential=False, postprocessing=postproc)
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [6]:
TEXT.build_vocab(trn, vectors='glove.6B.50d', max_size=vocab_size)
LABEL.build_vocab(trn)

loading vectors from .vector_cache/glove.6B.50d.pt


In [7]:
trn_iter, test_iter = data.Iterator.splits((trn, test), batch_size=64)
pad=TEXT.vocab.stoi['<pad>']
trn_td,test_td = (create_td(trn_iter),create_td(test_iter))

In [ ]:
from torchtext import vocab
glove = vocab.GloVe(name='twitter.27B', dim=50)
cng = vocab.CharNGram()

In [ ]:
class EzLSTM(nn.LSTM):
    def __init__(self, input_size, hidden_size, *args, **kwargs):
        super().__init__(input_size, hidden_size, *args, **kwargs)
        self.num_dirs = 2 if self.bidirectional else 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x):
        h0 = c0 = Variable(torch.zeros(self.num_dirs,x.size(1),self.hidden_size)).cuda()
        outp,_ = super().forward(x, (h0,c0))
        return outp[-1]

In [ ]:
def init_wgts(m, last_l=-2):
    c = list(m.children())
    for l in c:
        if isinstance(l, nn.Embedding): 
            l.weight.data.uniform_(-0.05,0.05)
        elif isinstance(l, (nn.Linear, nn.Conv1d)):
            xavier_uniform(l.weight.data, gain=calculate_gain('relu'))
            l.bias.data.zero_()
    xavier_uniform(c[last_l].weight.data, gain=calculate_gain('linear'));

class SeqSize(nn.Sequential):
    def forward(self, x):
        for l in self.children():
            x = l(x)
            print(x.size())
        return x

In [ ]:
class FastText(nn.Module):
    def __init__(self, nf, n_emb):
        super().__init__()
        self.emb = nn.Embedding(nf+1, n_emb, padding_idx=0)
        self.r = nn.Embedding(nf+1, 1)
        self.lin = nn.Linear(n_emb, 1)
        
    def forward(self, feat_idx, feat_cnt, sz):
        x = self.emb(feat_idx)
        r = self.r(feat_idx)
        x = x * r.abs()
        x = x.sum(1)/sz[:,None].float()
        x = self.lin(x)
        return x

m = FastText(vocab_size, 5).cuda()
m.r.weight.data = tr.unsqueeze(1).cuda()
m.r.weight.requires_grad = False
opt = optim.Adam(trainable_params_(m), 0.02, weight_decay=1e-8)

In [ ]:
fit(m, md, 2, F.binary_cross_entropy_with_logits, opt, metrics=[accuracy_thresh(0.5)])

In [ ]:
class FastText(nn.Module):
    def __init__(self, nf, n_emb):
        super().__init__()
        self.emb = nn.Embedding(nf+1, n_emb, padding_idx=0)
        self.conv1 = nn.Conv1d(n_emb, n_emb, 1)
        self.conv2 = nn.Conv1d(n_emb, n_emb, 2, padding=1)
        self.conv3 = nn.Conv1d(n_emb, n_emb, 3, padding=1)
        self.lin = nn.Linear(n_emb*2, 1)
        self.drop = nn.Dropout(0.01)
        self.bn1 = nn.BatchNorm1d(n_emb)
        self.bn2 = nn.BatchNorm1d(n_emb*2)
        
    def forward(self, feat_idx, feat_cnt, sz):
        x = self.emb(feat_idx)
        x = x.transpose(1,2).contiguous()
        x = self.bn1(x)
        x1 = self.conv1(x).sum(2)/sz[:,None].float()
        x2 = self.conv2(x).sum(2)/sz[:,None].float()
#         x3 = self.conv3(x).sum(2)/sz[:,None].float()
        x = torch.cat((x1,x2), 1)
        x = self.drop(self.bn2(F.relu(x)))
        x = self.lin(x)
        return x

m = FastText(vocab_size, 50).cuda()
# opt = optim.Adagrad(m.parameters(), 0.1)#, weight_decay=1e-6)
opt = optim.Adam(m.parameters(), 1e-2)

In [ ]:
p=0.1
# m = SeqSize(
m = nn.Sequential(
    nn.Embedding(nv, n_emb, padding_idx=pad),#, sparse=True), # S,B,C
    Lambda(lambda x: x.permute(1,2,0).contiguous()),
    nn.BatchNorm1d(n_emb),
    nn.MaxPool1d(2),
    nn.Conv1d(n_emb, n_emb*2, 5, padding=2),#, stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(n_emb*2),
    nn.MaxPool1d(2),
    nn.Dropout(p),
    nn.Conv1d(n_emb*2, n_emb*2, 5, padding=2),#, stride=2),
    nn.ReLU(),
    nn.BatchNorm1d(n_emb*2),
    nn.MaxPool1d(sl//4), # B,C,S
    nn.Dropout(p),
    Lambda(lambda x: x.squeeze()),
    nn.Linear(n_emb*2, 1),
    Lambda(lambda x: x.squeeze())
).cuda()

init_wgts(m)

In [ ]:
def fasttext_file(src):
    texts = []
    for name in ['neg','pos']:
        path = os.path.join(src, name)
        for fname in sorted(os.listdir(path)):
            fpath = os.path.join(path, fname)
            lines = [l for l in open(fpath)]
            texts.append(" ".join(lines) + f" __label__{name}\n")
    print(f"{src}/imdb_fast.txt")
    with open(f"{src}/imdb_fast.txt", "w") as f:
        f.writelines(texts)

In [ ]:
fasttext_file(f"{PATH}train")

In [ ]:
fasttext_file(f"{PATH}test")

In [ ]:
!wc -l {PATH}train/imdb_fast.txt

In [ ]:
import fasttext

In [ ]:
p = 12500
n = 12500

def accuracy(result, p, n):
    tp = result.recall * p
    fp = tp/result.precision - tp
    fn = tp/result.recall - tp
    tn = n - fp
    return (tp + tn)/(p + n)

In [ ]:
classifier = fasttext.supervised('tmp/aclImdb/train/imdb_fast.txt', 'model', silent=0,
                                 lr=0.5, epoch=10, word_ngrams=2)

In [ ]:
result = classifier.test("tmp/aclImdb/test/imdb_fast.txt")

In [ ]:
result.precision, result.recall

In [ ]:
accuracy(result, 12500, 12500)

### End